In [1]:
!pip install langchain
!pip install sentence_transformers
!pip install faiss-cpu
!pip install unstructured
!pip install "unstructured[pdf]"
!apt-get install -y poppler-utils
!pip install pdf2image
!pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e5623e97e6ae91b1bde1ba000dba5f249d17f05c5acd3cf33a5dcc77664b9363
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 70.7 MB/s

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 1s (367 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [1]:
from unstructured.partition.auto import partition
from unstructured.chunking.title import chunk_by_title
from sentence_transformers import SentenceTransformer,util
import faiss
from io import BytesIO
from typing import Tuple, List
import numpy as np
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores.faiss import FAISS
import torch
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, ContentSettings
device = torch.device('cuda'if torch.cuda.is_available() else'cpu')
print(device)

cuda


In [2]:
def getEmbeddings(text,model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(text)
    return embeddings

In [3]:
class Document:
  def __init__(self, page_content,embedding,  metadata,id):
      self.page_content = page_content
      self.embedding = embedding
      self.metadata = metadata
      self.id = id

In [4]:
# Function to embed and store chunks in vector database
def embed_and_store_chunks(folder_path):
    # Initialize the SentenceTransformer model
    model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

    # Create a list to store Document objects
    doc_chunks = []
    for filename in os.listdir(folder_path):
      if filename.endswith('pdf'):
        pdf_path = os.path.join(folder_path, filename)
        try:
          elements = partition(pdf_path)
        except Exception as e:
          print(f"An error occurred when trying to partition the file: {e,filename}")
          continue
        chunks = chunk_by_title(elements, new_after_n_chars=1500, combine_text_under_n_chars=700)
        # Embed each chunk and create Document objects
        for i, chunk in enumerate(chunks):
            # Embed the chunk using SentenceTransformer
            chunk_embedding = model.encode(chunk.text)
            page_number = chunk.metadata.page_number
            doc_id = f"{filename}-{i}"
            # Create a Document object for the chunk
            doc = Document(page_content=chunk.text, embedding=chunk_embedding ,metadata={"page": page_number},id=doc_id)
            doc.metadata["source"] = f"{doc.metadata['page']}-{doc_id}"
            doc.metadata["filename"] = filename
            doc_chunks.append(doc)
        print(filename,"complete")
    metadatax = [doc.metadata for doc in doc_chunks]  # Extract metadata from each Document
    idx = [doc.id for doc in doc_chunks]  # Extract ID from each Document
    # Extract just the embeddings from each Document
    embeddings = [model.encode(doc.page_content) for doc in doc_chunks]
    #Stores all encoded embeddings in the vector DB
    vectorstore_faiss = FAISS.from_embeddings([(doc.page_content, model.encode(doc.page_content)) for doc in doc_chunks], model,metadatas=metadatax,ids=idx)
    return vectorstore_faiss

# Embed and store chunks in vector database
vector_db = embed_and_store_chunks("/content/test/")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Bamboo  Bamboo Plantation.pdf complete


In [ ]:
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [ ]:
# from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from faiss import IndexFlatL2

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

text="dummy"
dimensions: int = len(embedding_function.embed_query(text))

index = IndexFlatL2(dimensions)
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

doc = Document(page_content=text, embedding=model.encode(text) ,metadata={"page": 0},id=0)

vector_db = FAISS(
    embedding_function=embedding_function,
    index=index,
    docstore = doc,
    index_to_docstore_id= 0,
)


In [5]:
def addPDFtoVectorDB(filepath,vector_db):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    doc_chunks = []
    if filepath.endswith('pdf'):
      filename = os.path.basename(filepath)
      print(filename,"started")
      elements = partition(filepath)
      # try:
      #   elements = partition(filepath)
      # except Exception as e:
      #   print(f"An error occurred when trying to partition the file: {e,filename}")
      chunks = chunk_by_title(elements, new_after_n_chars=1500, combine_text_under_n_chars=700)
      for i, chunk in enumerate(chunks):
            # Embed the chunk using SentenceTransformer
            chunk_embedding = model.encode(chunk.text)
            page_number = chunk.metadata.page_number
            doc_id = f"{filename}-{i}"
            # Create a Document object for the chunk
            doc = Document(page_content=chunk.text, embedding=chunk_embedding ,metadata={"page": page_number},id=doc_id)
            doc.metadata["source"] = f"{doc.metadata['page']}-{doc_id}"
            doc.metadata["filename"] = filename
            doc_chunks.append(doc)
      print(filename,"complete")
      metadatax = [doc.metadata for doc in doc_chunks]  # Extract metadata from each Document
      idx = [doc.id for doc in doc_chunks]  # Extract ID from each Document
      # Extract just the embeddings from each Document
      embeddings = [model.encode(doc.page_content) for doc in doc_chunks]
      # Add the embeddings to the vectorstore
      vector_db.add_embeddings(list(zip([doc.page_content for doc in doc_chunks], embeddings)), metadatas=metadatax, ids=idx)
    return vector_db

folder_path = "/content/"
for filename in os.listdir(folder_path):
   if filename.endswith('.pdf'):
       pdf_path = os.path.join(folder_path, filename)
       vector_db = addPDFtoVectorDB(filepath=pdf_path, vector_db=vector_db)

Lacquered Dreams - Promoting Livelihoods Through Lac.pdf started
Lacquered Dreams - Promoting Livelihoods Through Lac.pdf complete
IPM in Brinjal .pdf started
IPM in Brinjal .pdf complete
Weaving Livelihoods - A Socio-Economic Impact Study of Special SGSY Tasar Projects in Bihar and Jharkhand.pdf started
Weaving Livelihoods - A Socio-Economic Impact Study of Special SGSY Tasar Projects in Bihar and Jharkhand.pdf complete
SELCO-Foundation_-65-livelihoods-solutions-from-ground.pdf started
SELCO-Foundation_-65-livelihoods-solutions-from-ground.pdf complete
Chick Pea - Value Chain analysis.pdf started
Chick Pea - Value Chain analysis.pdf complete
FPC registration procedure.pdf started
FPC registration procedure.pdf complete
FINGER MILLET - PPT - trg by Ganapathy KN .pdf started
FINGER MILLET - PPT - trg by Ganapathy KN .pdf complete
Lac - Value Chain Analysis Report.pdf started
Lac - Value Chain Analysis Report.pdf complete


In [6]:
Query = "what is chick pea"

e2 = getEmbeddings(Query)
k = 20
top_k_result =vector_db.similarity_search_by_vector(e2,k)

for i in range(k):
  print(top_k_result[i].page_content)
  print(top_k_result[i].metadata)
  # print(top_k_result[i].si)
  print("-------------------------------------------------------------------------------")

Chick Pea: Value Chain Analysis

storage facilities for farmers at a cluster level which will help them in realising 15% more

farm gate prices. It can be housed or nested in a farmers’ cooperative or collective

SEGMENT E (UN-IRRIGATED WHEAT DIET BASED FAMILIES)

This is a special and a very unique segment which owing to very poor quality soils and lack of

irrigation infrastructure go for Kharif fallows and then a chickpea crop in the Rabi season. These
{'page': 32, 'source': '32-Chick Pea - Value Chain analysis.pdf-123', 'filename': 'Chick Pea - Value Chain analysis.pdf'}
-------------------------------------------------------------------------------
Chick Pea: Value Chain Analysis

in the Bronze Age, most popularly, Italy and Greece. Those people consumed chickpea in various

forms like roasted as snacks, raw, carbonized or in broth. Many past writings have also been found

telling the uses and importance of chickpeas both medical and as a food crop.

The ancient wisdom of the Indi

In [7]:
def saveVectorDB(vector_db,path):
    vector_db.save_local(path)

def loadVectorDB(path,model='all-MiniLM-L6-v2'):
    vector_db = FAISS.load_local(path,model)
    return vector_db

saveVectorDB(vector_db,"faiss_index")

In [8]:
account_name = "ragvectordbcontainer"
account_key =""
container_name ="vecctordbcontainer1"

In [11]:

def upload_folder_to_blob(account_name, account_key, container_name, local_folder_path, remote_folder_name):
    # Create a connection string
    connect_str = f'DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net'

    # Create a BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Get a container client
    container_client = blob_service_client.get_container_client(container_name)

    # Iterate through local files and upload them to Azure Blob Storage
    for root, dirs, files in os.walk(local_folder_path):
        for file_name in files:
            local_file_path = os.path.join(root, file_name)
            blob_name = os.path.join(remote_folder_name, os.path.relpath(local_file_path, local_folder_path)).replace("\\", "/")

            # Create a BlobClient and upload the file
            blob_client = container_client.get_blob_client(blob_name)
            with open(local_file_path, "rb") as data:
                blob_client.upload_blob(data, overwrite=True)

    print(f"Folder '{local_folder_path}' uploaded to Azure Blob Storage container '{container_name}/{remote_folder_name}'.")


upload_folder_to_blob(account_name, account_key, container_name, "/content/faiss_index", "faiss_index")


Folder '/content/faiss_index' uploaded to Azure Blob Storage container 'vecctordbcontainer1/faiss_index'.


In [12]:

def download_folder_from_blob(account_name, account_key, container_name, remote_folder_name, local_folder_path):
 # Create a connection string
 connect_str = f'DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net'

 # Create a BlobServiceClient
 blob_service_client = BlobServiceClient.from_connection_string(connect_str)

 # Get a container client
 container_client = blob_service_client.get_container_client(container_name)

 # List blobs in the specified folder
 blob_list = container_client.list_blobs(name_starts_with=remote_folder_name)

 # Download each blob to the local folder
 for blob in blob_list:
     blob_name = blob.name

     # Extract the base file name from the blob name
     base_file_name = os.path.basename(blob_name)

     local_file_path = os.path.join(local_folder_path, base_file_name)

     # Create necessary directories
     os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

     # Create a BlobClient and download the file
     blob_client = container_client.get_blob_client(blob_name)
     with open(local_file_path, "wb") as data:
         data.write(blob_client.download_blob().readall())

 print(f"Folder '{container_name}/{remote_folder_name}' downloaded to '{local_folder_path}'.")
download_folder_from_blob(account_name, account_key, container_name, "faiss_index", "/content/faiss_index2")


Folder 'vecctordbcontainer1/faiss_index' downloaded to '/content/faiss_index2'.


In [13]:
loaded_VDB = loadVectorDB("/content/faiss_index2")

In [14]:
Query = "what is chick pea"

e2 = getEmbeddings(Query)
k = 20
top_k_result =loaded_VDB.similarity_search_by_vector(e2,k)

for i in range(k):
  print(top_k_result[i].page_content)
  print(top_k_result[i].metadata)
  # print(top_k_result[i].si)
  print("-------------------------------------------------------------------------------")

Chick Pea: Value Chain Analysis

storage facilities for farmers at a cluster level which will help them in realising 15% more

farm gate prices. It can be housed or nested in a farmers’ cooperative or collective

SEGMENT E (UN-IRRIGATED WHEAT DIET BASED FAMILIES)

This is a special and a very unique segment which owing to very poor quality soils and lack of

irrigation infrastructure go for Kharif fallows and then a chickpea crop in the Rabi season. These
{'page': 32, 'source': '32-Chick Pea - Value Chain analysis.pdf-123', 'filename': 'Chick Pea - Value Chain analysis.pdf'}
-------------------------------------------------------------------------------
Chick Pea: Value Chain Analysis

in the Bronze Age, most popularly, Italy and Greece. Those people consumed chickpea in various

forms like roasted as snacks, raw, carbonized or in broth. Many past writings have also been found

telling the uses and importance of chickpeas both medical and as a food crop.

The ancient wisdom of the Indi